In [1]:
# Importing the libraries
import datetime
import h5py
from selenium import webdriver
import time
import pandas as pd

# Scraping

In [2]:
# Setting up
date = datetime.date.today()
browser = webdriver.Chrome(r'C:\Users\FILI DANI\Downloads\chromedriver_win32\chromedriver.exe')
browser.maximize_window()
website = pd.read_csv('websites.csv')

In [3]:
# Worldometers website data retrieval 
browser.get(f'{website.iloc[0,1]}')
time.sleep(8)
country_names_list = []
country_population_list = []
content = browser.find_element_by_xpath('//div[@class="content"]/div[@class="row"]')
country_names = content.find_elements_by_class_name('t20-country')
country_population = content.find_elements_by_class_name('t20-number')
for country in country_names:
    country_names_list.append(country.text)
for population in country_population:
    country_population_list.append(population.text)

In [4]:
# Cleaning worldometers data
country_population_list_clean = []
for population in country_population_list:
    country_population_list_clean.append(int(population.replace(',','')))

In [5]:
# Trading view website data retrieval
browser.get(f'{website.iloc[1,1]}')
time.sleep(80)
company_names_list = []
company_prices_list = []
content = browser.find_element_by_xpath('//div[@class="container-aPcTuZxR"]')
company_names = content.find_elements_by_class_name('titleContainer-gVvYQyhR')
company_prices = content.find_elements_by_xpath('.//span[@class="last-gVvYQyhR js-symbol-last"]')
for company_name in company_names:
    company_names_list.append(company_name.text)
for company_price in company_prices:
    company_prices_list.append(company_price.text)

In [6]:
# Cleaning trading view data
company_prices_list_clean = []
for company_price in company_prices_list:
    company_prices_list_clean.append(float(company_price))

In [7]:
# Coin market cap website data retrieval
browser.get(f'{website.iloc[2,1]}')
time.sleep(10)
crypto_currency_names_list = []
crypto_currency_marketcaps_list = []
browser.execute_script("window.scrollTo(0, 1100);")
time.sleep(25)
content = browser.find_element_by_tag_name('table')
crypto_currency_names = content.find_elements_by_xpath('//div[@class="sc-aef7b723-0 sc-8497df48-0 iULUNk"]/div/p')
crypto_currency_marketcaps = content.find_elements_by_xpath('//tbody/tr/td/p[@class="sc-4984dd93-0 jZrMxO"]')
for crypto_currency_name in crypto_currency_names:
    crypto_currency_names_list.append(crypto_currency_name.text)
for crypto_currency_marketcap in crypto_currency_marketcaps:
    crypto_currency_marketcaps_list.append(crypto_currency_marketcap.text)

In [8]:
# Cleaning coin market cap data
crypto_currency_marketcaps_list_clean_1 = []
crypto_currency_marketcaps_list_clean_2 = []
for crypto_currency_marketcap in crypto_currency_marketcaps_list:
    crypto_currency_marketcaps_list_clean_1.append(crypto_currency_marketcap.replace('$',''))
for crypto_currency_marketcap in crypto_currency_marketcaps_list_clean_1:
    crypto_currency_marketcaps_list_clean_2.append(float(crypto_currency_marketcap.replace(',','')))

In [9]:
# Saving the data
with h5py.File('web-scraping','a') as f:
    world_population_dataset = f['World population'].create_dataset(f'World population {date}',shape=(20,),dtype=[('country names','S50'),('country population',int)])
    investment_dataset = f['Investment'].create_dataset(f'Investment {date}',shape=(6,),dtype=[('company names','S50'),('company prices',float)])
    crypto_currency_dataset = f['Crypto currency'].create_dataset(f'Crypto currency {date}',shape=(30,),dtype=[('crypto currency names','S50'),('crypto currency marketcaps',float)])
    world_population_dataset['country names'] = country_names_list
    world_population_dataset['country population'] = country_population_list_clean
    investment_dataset['company names'] = company_names_list
    investment_dataset['company prices'] = company_prices_list_clean
    crypto_currency_dataset['crypto currency names'] = crypto_currency_names_list
    crypto_currency_dataset['crypto currency marketcaps'] = crypto_currency_marketcaps_list_clean_2
    f.close()